# Minitarea 2

Nombre: Miguel Videla A.

---------------------------
## Language Models (3 pts)
Estas preguntas son teóricas y deben ser resueltas con desarrollo, pero sin código. Por favor usen $\LaTeX$ para las fórmulas. El material del curso y los videos deberian ser suficientes para que puedan responder.


### Pregunta 1 (1 pt)
Asuma que tiene un modelo de lenguaje de trigramas (simple) entrenado sobre un corpus C. Muestre cómo el modelo le asigna una probabilidad a la oración `el perro le ladra al gato` usando los parámetros estimados a partir del corpus  $q(w_i | w_{i-2}, w_{i-1})$ .

**Respuesta:**

Sea $\mathcal{C}$ un conjunto finito de palabras, el modelo de lenguaje de trigramas queda definido como un **proceso de Markov a tiempo discreto de segundo orden**, es decir, dado un proceso estocástico a tiempo discreto $X=(X_n)_{n\in \mathbb{N}}$, $\forall w_0,w_1,...,w_n\in \mathcal{C}$ se satisface la propiedad de Markov de segundo orden:
\begin{equation}
P(X_{n+1}=w_{n+1}|X_0=w_0,X_1=w_1,...,X_n=w_n)=P(X_{n+1}=w_{n+1}|X_n=w_n,X_{n-1}=w_{n-1})
\label{eq:markov} \tag{1}
\end{equation}

Sea $(\text{'el','perro','le','ladra','al','gato'})=(w_1,w_2,w_3,w_4,w_5)$, la probabilidad de ocurrencia de la oración "*el perro le ladra al gato*" $q(w_1,w_2,w_3,w_4,w_5)$, dada la propiedad de regla de la cadena en probabilidades y la propiedad (1), queda determinada por:
\begin{equation}
q(w_1,w_2,w_3,w_4,w_5)=\prod_{i=1}^{5}q(w_i|w_{i-2},w_{i-1})
\label{eq:trigram} \tag{2}
\end{equation}

con $w_0=w_{-1}=*$. Se observa que el modelo debe ser capaz de estimar probabilidades de secuencias de palabras de largo variable, por lo que dada la probabilidad de un trigrama $q(u|v,w)$, se tiene que $u\in \mathcal{C} \cup \{STOP\}$ y $v,w\in \mathcal{C}\cup \{*\}$, con $STOP$ el símbolo de término de una secuencia de palabras y $*$ el símbolo de inicio de secuencia, tal que $\forall i\in\{1,2,...,|\mathcal{C}|\}$, $q(w_{i}|*,w_{i-1})=q(w_{i}|w_{i-1})$ y $q(w_{i}|*,*)=q(w_{i})$.

Finalmente, la estimación empírica de las probabilides condicionales sobre $\mathcal{C}$ se calcula como:
\begin{equation}
\begin{split}
\hat{q}_{ML}(w_i|w_{i-2},w_{i-1})&=\frac{count(w_i,w_{i-1},w_{i-2})}{count(w_{i-1},w_{i-2})}\\
\hat{q}_{ML}(w_i|w_{i-1})&=\frac{count(w_i,w_{i-1})}{count(w_{i-1})}\\
\hat{q}_{ML}(w_i)&=\frac{count(w_i)}{|\mathcal{C}|}
\end{split}
\label{eq:ml} \tag{3}
\end{equation}

los cuales corresponden a la **estimación de máxima verosimilitud** de los modelos trigramas, bigramas y unigramas, respectivamente.

### Pregunta 2 (1 pt)
Muestre cómo se calcularía  $q(w_{i} | w_{i-2}, w_{i-1})$  usando un modelo que interpola un modelo de lenguajes de trigramas, bigramas, y unigrama.

**Respuesta:**

El modelo interpolado corresponde a una **suma ponderada de las estimaciones de máxima verosimilitud** de cada modelo (3). De este modo, la probabilidad de $q(w_{i} | w_{i-2}, w_{i-1})$ del modelo ponderado resulta:
\begin{equation}
q(w_{i} | w_{i-2}, w_{i-1}) = \lambda_1\cdot \hat{q}_{ML}(w_{i} | w_{i-2}, w_{i-1})+\lambda_2\cdot \hat{q}_{ML}(w_{i} | w_{i-1})+\lambda_3\cdot \hat{q}_{ML}(w_{i})
\label{eq:interpolation_model} \tag{4}
\end{equation}

con $\lambda_j\geq 0$, $\forall j\in\{1,2,3\}$ y $\lambda_1+\lambda_2+\lambda_3=1$.

Los valores $\lambda_j$ se obtienen mediante la resolución del siguinte problema de optimización:
\begin{equation}
(\lambda_1^*,\lambda_2^*,\lambda_3^*)=\underset{\lambda_1,\lambda_2,\lambda_3}{argmax} \sum_{w_1,w_2,w_3\in\mathcal{C}_{validation}}c'(w_1,w_2,w_3)\log q(w_{i} | w_{i-2}, w_{i-1})
\label{eq:L_lambda} \tag{5}
\end{equation}

sujeto a las restricciones anteriores, con $\mathcal{C}=\mathcal{C}_{train} \dot\cup \mathcal{C}_{validation}$, $c'(w_1,w_2,w_3)$ la cantidad de ocurrencias de $(w_1,w_2,w_3)$ en $\mathcal{C}_{validation}$ y $\mathcal{C}_{train}$ el conjunto utilizado para estimar las probabilidades en (3).

### Pregunta 3 (1 pt)
¿Qué ventajas tiene el modelo interpolado sobre el modelo de trigramas simple?

**Respuesta:**

Dependiendo del corpus utilizado será preferible utilizar algún respectivo modelo n-grama. El modelo interpolado permite incorporar de manera óptima los distintos modelos n-gramas, ponderando la predicción de cada uno de acuerdo al conjuto de validación presentado, con el objetivo de minimizar la *perplexity* del modelo, y por tanto, robusteciendo su desempeño frente a nuevas muestras de *test*. Una menor *perplexity* implica una mayor certeza en la predicción de ejemplos de *test*, asignándole mayor probabilidad (el modelo está menos sorprendido al presentarle nuevas muestras a predecir).


-----------------------
## Naive Bayes (3 pts)
En esta parte de la minitarea deberan programar Naive Bayes Multinomial usando Laplace Smothing. Las referencias las pueden encontrar en el material del curso y los videos del profesor.

A continuacion se presentan un conjunto de documentos de training divididos en 2 categorias distintas. Ustedes deberan clasificar los documentos del test set usando Naive Bayes con Laplace Smothing.

Por favor, documenten su codigo. Escriban que hacen las funciones, que reciben, que entregan, etc. Si en algun lugar de su codigo hacen algo que creen que debe ser explicado, pongan comentarios, son bienvenidos. 


**Underflow prevention:** En vez de hacer muchas multiplicacions de `float`s, reemplacenlas por sumas de logaritmos para prevenir errores de precision. Revisen la diapo 69 de las slides. 

NOTA: Sobre las `namedtuple`s. Es el tipo de los documentos. Son objetos inmutable que tienen dos atributos: `words` donde estan las palabras del documento y `class_` donde se guarda la clase de ese documento. Estos objetos son inmutables, lo que quiere decir que si quieren modificar un documento y cambiarle la clase, tienen que crear otro documento. Otra cosa es que son tuplas como cualquier otra, es decir se pueden acceder usando indices como `doc[0]` o `doc[1]`. Son libres de implementar su solucion como quieran, si no les gusta este tipo de dato usen otro.



In [1]:
from pprint import pprint
from collections import namedtuple
document = namedtuple(
    "document", ("words", "class_")  # avoid python's keyword collision
)

train_set = (
    document(words=('w03', 'w01', 'w02', 'w06', 'w02', 'w08', 'w07'), class_=0),
    document(words=('w05', 'w04', 'w00', 'w06', 'w09', 'w07', 'w06', 'w09', 'w05'), class_=1),
    document(words=('w07', 'w06', 'w00', 'w08', 'w01', 'w08', 'w08', 'w09', 'w02'), class_=0),
    document(words=('w08', 'w09', 'w02', 'w06', 'w05', 'w08', 'w07'), class_=1),
    document(words=('w09', 'w08', 'w05', 'w08', 'w05', 'w00', 'w08'), class_=1),
    document(words=('w05', 'w05', 'w06', 'w01', 'w06', 'w08', 'w02'), class_=1),
    document(words=('w04', 'w03', 'w07', 'w05', 'w04', 'w00', 'w02'), class_=0),
    document(words=('w01', 'w00', 'w01', 'w04', 'w09', 'w02', 'w04', 'w07'), class_=1)
)
print("Train documents:")
pprint(train_set)


test_set = (document(words=('w02', 'w09', 'w06', 'w01', 'w05', 'w04', 'w03', 'w03'), class_=None),)
print("\nTest documents:")
pprint(test_set)

Train documents:
(document(words=('w03', 'w01', 'w02', 'w06', 'w02', 'w08', 'w07'), class_=0),
 document(words=('w05', 'w04', 'w00', 'w06', 'w09', 'w07', 'w06', 'w09', 'w05'), class_=1),
 document(words=('w07', 'w06', 'w00', 'w08', 'w01', 'w08', 'w08', 'w09', 'w02'), class_=0),
 document(words=('w08', 'w09', 'w02', 'w06', 'w05', 'w08', 'w07'), class_=1),
 document(words=('w09', 'w08', 'w05', 'w08', 'w05', 'w00', 'w08'), class_=1),
 document(words=('w05', 'w05', 'w06', 'w01', 'w06', 'w08', 'w02'), class_=1),
 document(words=('w04', 'w03', 'w07', 'w05', 'w04', 'w00', 'w02'), class_=0),
 document(words=('w01', 'w00', 'w01', 'w04', 'w09', 'w02', 'w04', 'w07'), class_=1))

Test documents:
(document(words=('w02', 'w09', 'w06', 'w01', 'w05', 'w04', 'w03', 'w03'), class_=None),)


In [2]:
import numpy as np

class MultinomialNaiveBayes():
    """Multinomial Naive Bayes language model with smoothing"""
    def __init__(self, smoothing_factor=0):
        """
        Initializes the class attributes

        Parameters
        ----------
        smoothing_factor : float
            Smoothing factor of estimated maximum likelihood probability 

        Returns
        -------
        None      
        """
        self.smoothing_factor = smoothing_factor
        self.docs = None
        self.docs_classes = None
        self.classes = None
    
    def fit(self, corpus):
        """
        Set the class attributes according to a corpus

        Parameters
        ----------
        corpus : tuple(document)
            A tuple with the documents(words, class_) of the corpus

        Returns
        -------
        None  
        """
        self.docs = []
        self.docs_classes = []
        for doc, class_ in corpus:
            self.docs.append(np.array(doc))
            self.docs_classes.append(class_)
        self.docs = np.array(self.docs)
        self.docs_classes = np.array(self.docs_classes)
        self.classes = np.unique(self.docs_classes)
            
    def get_vocabulary(self, class_=None, return_counts=False):
        """
        Return the vocabulary according to the documents array class attribute

        Parameters
        ----------
        class_ : int, optional
            If is not None returns the vocabulary of the documents array class attribute of the given class
        return_counts : bool, optional
            If true return a tuple with vocabulary and the number of ocurrences

        Returns
        -------
        array(str)
            Corpus vocabulary
        """
        vocab = []
        for i in range(len(self.docs)):
            if (self.docs_classes[i] == class_) or (class_ is None):
                vocab += list(self.docs[i])
        return np.unique(vocab, return_counts=return_counts)
        
    def prior(self, class_):
        """
        Calculates the estimated prior probability

        Parameters
        ----------
        class_ : int
            A class of corpus documents

        Returns
        -------
        float
            Number of documents of a class in the corpus divided by the total number of documents
        """
        return (self.docs_classes == class_).sum() / len(self.docs)
    
    def likelihood(self, word, class_):
        """
        Calculates the estimated likelihood probability

        Parameters
        ----------
        word : str
            An input word
        class_ : int
            A class of corpus documents

        Returns
        -------
        float
            Number of occurences of the input word in the corpus divided by the number of words in the 
            documents of the corpus with class equals to the input class (smoothed)
        """
        vocab = self.get_vocabulary()
        class_vocab, class_ocurrences = self.get_vocabulary(class_, True)
        ocurrence_idxs = np.argwhere(class_vocab == word)
        if len(ocurrence_idxs) == 0:
            word_ocurrences = 0
        else:   
            word_ocurrences = class_ocurrences[ocurrence_idxs[0][0]]
        return (word_ocurrences + self.smoothing_factor) / (class_ocurrences.sum() + self.smoothing_factor * len(vocab))
    
    def log_posterior(self, doc, class_):
        """
        Calculates the estimated log-posterior probability according to the Bayes theorem

        Parameters
        ----------
        doc : array(str)
            Input document
        class_ : int
            A class of corpus documents

        Returns
        -------
        float
            Class log prior plus sum of log likelihood of each word in document
        """
        log_post = np.log(self.prior(class_))
        for word in doc:
            log_post += np.log(self.likelihood(word, class_))
        return log_post
    
    def class_prediction(self, doc):
        """
        Return the class prediction of a given document according to the maximum a posteriri estimation

        Parameters
        ----------
        doc : array(str)
            Input document

        Returns
        -------
        int
            Predicted class
        """
        return self.classes[np.argmax([self.log_posterior(doc, class_) for class_ in self.classes])]

    def predict(self, doc_tuples):
        """
        Return the class predictions of each document of the input corpus

        Parameters
        ----------
        doc_tuples : tuple(documents)
            Input document

        Returns
        -------
        array(int)
            Predicted class for each document
        """
        return np.array([self.class_prediction(doc_tuple[0]) for doc_tuple in doc_tuples])

# Class initialization
nb = MultinomialNaiveBayes(smoothing_factor=1)
# Training corpus set fitting
nb.fit(train_set)
# Test corpus set prediction
predictions = nb.predict(test_set)
print("Predictions:\n{}".format(predictions))

Predictions:
[0]
